<a href="https://colab.research.google.com/github/DawSkiii/AI/blob/main/AI_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DE PERALTA, RON MARI FLEUR L.
CARINO, JOHN JERVIN P.
CALIBOSO, BEN FRANKLIN S.
CABAIS, JEFREN PAUL A.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

print("TensorFlow version:", tf.__version__)

In [ ]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
df = pd.read_csv(url)
df['Month'] = pd.to_datetime(df['Month'])
df.set_index('Month', inplace=True)

df.head()

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(df.index, df['Passengers'])
plt.title("International Airline Passengers")
plt.xlabel("Date")
plt.ylabel("Passengers")
plt.grid(True)
plt.show()

In [ ]:
values = df['Passengers'].values.reshape(-1,1)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(values)

def create_sequences(data, look_back=12):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back])
        y.append(data[i+look_back])
    return np.array(X), np.array(y)

LOOK_BACK = 12
X, y = create_sequences(scaled, LOOK_BACK)

X.shape, y.shape

In [ ]:
TEST_SIZE = 12  # last 12 months for testing
X_train, X_test = X[:-TEST_SIZE], X[-TEST_SIZE:]
y_train, y_test = y[:-TEST_SIZE], y[-TEST_SIZE:]

X_train.shape, X_test.shape

In [ ]:
model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(LOOK_BACK, 1)),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=8,
    validation_data=(X_test, y_test),
    verbose=2,
    callbacks=[EarlyStopping(patience=15, restore_best_weights=True)]
)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
y_pred_scaled = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test.reshape(-1,1))
y_pred_inv = scaler.inverse_transform(y_pred_scaled)

y_test_inv[:5], y_pred_inv[:5]

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
rmse

In [ ]:
dates_test = df.index[-TEST_SIZE:]

plt.figure(figsize=(10,4))
plt.plot(df.index, df['Passengers'], label="Original Full Series")
plt.plot(dates_test, y_test_inv, label="True")
plt.plot(dates_test, y_pred_inv, label="Predicted")
plt.title("LSTM — True vs Predicted")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
future_steps = 12
last_seq = scaled[-LOOK_BACK:].copy().reshape(1, LOOK_BACK, 1)

future_predictions = []

current_seq = last_seq.copy()
for _ in range(future_steps):
    pred = model.predict(current_seq)[0,0]
    future_predictions.append(pred)
    current_seq = np.append(current_seq[:, 1:, :], [[[pred]]], axis=1)

future_inv = scaler.inverse_transform(np.array(future_predictions).reshape(-1,1))
future_inv

In [ ]:
future_dates = pd.date_range(df.index[-1] + pd.offsets.MonthBegin(1), periods=future_steps, freq="MS")

plt.figure(figsize=(10,4))
plt.plot(df.index, df['Passengers'], label="Historical")
plt.plot(future_dates, future_inv, label="Forecast", marker='o')
plt.title("12-Month Future Forecast")
plt.grid(True)
plt.legend()
plt.show()